In [1]:
import numpy as np

In [ ]:
def forward(pi, a, b, x):
    T = x.shape[0]
    alpha = np.zeros((T, 2), np.float64)
    # alpha[t][i]
    alpha[0][0] = pi[0]*b[0][x[0]]
    alpha[0][1] = pi[1]*b[1][x[0]]
    
    for t in range(1,T):
        alpha[t] = b[:,x[t]] * np.sum(a.T * alpha[t-1], axis = 1)
    alpha += 1e-35
    return alpha
    
def backward(a, b, x):
    T = x.shape[0]
    beta = np.zeros((T, 2))
    # beta[t][i]
    beta[T-1][0] = 1
    beta[T-1][1] = 1
    
    for t in reversed(range(1, T)):
        beta[t-1] = np.sum(beta[t] * a * b[:, x[t]], axis = 1)
    beta += 1e-35
    return beta

def baum_welch(alpha, beta, a, b, x):
    T = x.shape[0]
    states = a.shape[0]
    
    # gamma[t][i]
    gamma = alpha * beta
    gamma /= np.sum(gamma, axis=1).reshape(T,1)
    
    # xi[t][i][j]
    xi = alpha.reshape(T,states,1)[0:T-1] * beta.reshape(T,1,states)[1:T] * (a * b.T[x[1:T]].reshape(T-1, 1, states))
    xi /= np.sum(xi, axis=(1,2)).reshape(T-1, 1, 1)
        
    
    # update a & b & pi
    a = np.sum(xi,axis = 0) / np.sum(gamma[0:T-1], axis = 0).reshape(states, 1)
    
    b = np.zeros((states, states))
    np.add.at(b.T, x, gamma)
    b /= np.sum(gamma, axis = 0).reshape(states, 1)
    
    
    pi = gamma[0]
    return pi, a, b